# Предсказание плохих или хороших ФИО : схема решения

В задаче рассматриваются ФИО зарегистрированных в платёжной системе пользователей.    

Каждому ФИО соответствует метка класса:    
1) 0 - ФИО хорошее;    
2) 1 - ФИО хорошее, но в нём присутствует опечатка;    
3) 2 - ФИО плохое (трешовые наименования).

Цель: предсказать метки наиболее точно.    

Данные:   
1) Непосредственно полное имя (необязательно состоящее из фамилии, имени и отчества, могут быть другие слова, может не быть отчества, т.д.);    
2) Код гражданства.

## Загрузка данных

In [1]:
import pandas as pd

In [5]:
train = pd.read_csv('fio_data/fio_train.csv')
test = pd.read_csv('fio_data/fio_test_raw.csv')

In [6]:
train.head()

,FULLNAME,COUNTRYISO,label
0,ЛЫКОВА ЮЛИЯ ВЛАДИМИРОВНА,RUS,0
1,КИРИЛИНА МАРИНА ИВАНОВНА,RUS,0
2,КОНЮХОВ ЕВГЕНИЙ ВИКТОРОВИЧ,RUS,0
3,ЧУРКИН АРТЁМ ВЯЧЕСЛАВОВИЧ,RUS,0
4,АЛИЕВ ГЮНДУЗ ХЫДЫРОГЛЫ,RUS,0


In [7]:
test.head()

,FULLNAME,COUNTRYISO
0,KARAPETYAN GEGAHAM,ARM
1,АЗИМОВ АХКОМИДИН АСОЕВИЧ,TJK
2,KHUSENOV ABDUMATIN,GEO
3,МИКИНА ДИАНА СЕРГЕЕВНА,RUS
4,ЛЕВАШОВ ВИКТОР АЛЕКСАНДРОВИЧ,RUS


##### Распределение меток на тестовой выборке

In [9]:
train['label'].value_counts()

0    764989
1    135013
2       154
Name: label, dtype: int64

## Построение матриц признаков

Для работы с классификаторами необходимо построить вектора признаков для каждого ФИО. Воспользуемся традиционным подходом для задач обработки естественного языка: "мешком" символьных триграмм.

Но сперва установим пакет, который позволит нам быстро вычислять индекс той или иной триграммы в массиве всех возможных триграмм

In [48]:
!pip install dawg

In [55]:
from dawg import IntDAWG
from scipy import sparse

Создаём список всех наших признаков

In [56]:
accepted_chars_rus = "АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ'.-"
accepted_chars_lat = "ABCDEFGHIJKLMNOPQRSTUVWXYZ'.-"
top_iso = [ 'iso_' + a for a in list(train['COUNTRYISO'].value_counts().keys())[:25] ]
features = list()
for i in accepted_chars_lat:
    for j in accepted_chars_lat:
        for k in accepted_chars_lat:
            features.append(i+j+k)
for i in accepted_chars_rus:
    for j in accepted_chars_rus:
        for k in accepted_chars_rus:
            features.append(i+j+k)
features.extend(top_iso)
features_pairs = list(zip(features, range(len(features))))
features_dawg = IntDAWG(features_pairs)
feature_list = list(accepted_chars_rus + accepted_chars_lat) + top_iso
trigrams = lambda a: zip(a, a[1:], a[2:])

In [57]:
def construct_datasets(row):
    cur_feat_ids = []
    for trigram in trigrams(row['FULLNAME']):
        trigram = ''.join(trigram)
        if trigram in feature_list:
            cur_feat_ids.append(features_dawg[trigram])
    iso = 'iso_' + row['COUNTRYISO']
    if iso in features_dawg:
        cur_feat_ids.append(features_dawg[iso])
    cur_feat_ids = sorted(list(set(cur_feat_ids)))
    data = np.array([1]*len(cur_feat_ids))
    indices = np.array(cur_feat_ids)
    indptr = np.array([0,len(cur_feat_ids)])
    return sparse.csr_matrix((data, indices, indptr), shape=(1, len(features))).astype(np.float64)

In [58]:
%%time
train['sparse_feats'] = train.apply(construct_datasets, axis=1)
test['sparse_feats'] = test.apply(construct_datasets, axis=1)

CPU times: user 3min 30s, sys: 1.07 s, total: 3min 31s
Wall time: 3min 31s


##### Сохраним матрицы признаков и метки в файлы, чтобы не пересчитывать их в дальнейшем

In [37]:
!mkdir features

mkdir: cannot create directory ‘features’: File exists


In [43]:
%%time
X_train = sparse.vstack(train['sparse_feats'].values)
y_train = fio_full_cleaned['label'].values
X_test = sparse.vstack(test['sparse_feats'].values)

CPU times: user 192 ms, sys: 1.32 s, total: 1.51 s
Wall time: 1.61 s


In [ ]:
sparse.save_npz('features/X_train.npz', X_train)
sparse.save_npz('features/X_test.npz', X_test)
with open('features/y_train.pkl', 'wb') as f:
    pickle.dump(y_train, f)

## Классификатор

In [63]:
import numpy as np

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

##### Оценка качества классификатора на кросс-валидации

In [75]:
%%time
cv_scores = cross_val_score(
    LogisticRegression(solver='lbfgs'),
    sparse.vstack(train['sparse_feats'].values),
    train['label'].values,
    scoring='f1_macro',
    cv=5,
)
print('Bad FIO classifier: f1_macro={}, cv_scores={}'.format(cv_scores.mean(), cv_scores))

/pub/home/ansec/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/pub/home/ansec/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/pub/home/ansec/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/pub/home/ansec/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Bad FIO classifier: f1_macro=0.3062752292873576, cv_scores=[0.30627471 0.30627471 0.30627471 0.30627563 0.30627639]
CPU times: user 25min 24s, sys: 5min 31s, total: 30min 56s
Wall time: 2min 2s


/pub/home/ansec/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Предупреждения говорят нам о том, что в каком-то одном классе ни один элемент не предсказался верно. Низкие показатели макро F-меры также нам говорят о том, что некоторые классы распознаются с низкой точностью и полнотой.    
Задача: предложить такие признаки и алгоритмы, которые будут иметь лучшие показатели макро F-меры.

##### Обучение классификатора и предсказание на тестовой выборке

In [76]:
%%time
model = LogisticRegression(solver='lbfgs').fit(X_train, y_train)

CPU times: user 16min 8s, sys: 17min 55s, total: 34min 3s
Wall time: 1min 27s


In [77]:
y_test = model.predict(X_test)

In [78]:
y_test

array([0, 0, 0, ..., 0, 0, 0])

## Файл с предсказаниями
Файл представляет собой список меток, которые были проставлены алгоритмом. Важно сохранить количество и порядок меток.

In [81]:
!mkdir predictions

In [82]:
with open('predictions/sample_submission.csv', 'w') as f:
    f.write('\n'.join(list(map(str, y_test))))

In [83]:
!head predictions/sample_submission.csv

0
0
0
0
0
0
0
0
0
0
